In [1]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [27]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [28]:
tools = [wiki, arxiv_tool]

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
groq_api_key = os.getenv('GROQ_API_KEY')

In [11]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding= HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(embedding=embedding, documents=docs)
retriever = vector_db.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriever, "Langchain Search", "searches the langchain official documentation page for any query")

In [29]:
tools = [wiki, arxiv_tool, retriver_tool]

In [19]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model='openai/gpt-oss-20b',
    groq_api_key=groq_api_key
)

In [16]:
from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [31]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [34]:
agent_executor.invoke({'input' : "what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML fi**Machine learning (ML)** is a subfield of artificial intelligence that focuses on developing statistical algorithms capable of learning patterns from data and generalizing to new, unseen data. Rather than being explicitly programmed for every task, ML systems automatically improve their performance by analyzing examples and adjusting their internal parameters. Recent advances in deep learning—p

{'input': 'what is machine learning',
 'output': '**Machine learning (ML)** is a subfield of artificial intelligence that focuses on developing statistical algorithms capable of learning patterns from data and generalizing to new, unseen data. Rather than being explicitly programmed for every task, ML systems automatically improve their performance by analyzing examples and adjusting their internal parameters. Recent advances in deep learning—particularly neural networks—have enabled ML models to achieve state‑of‑the‑art results in many areas, often surpassing earlier machine‑learning techniques.'}

In [35]:
agent_executor.invoke({'input' : 'what is the abstract of Attentio is all you need?'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention Is All You Need abstract'}`


Page: Kevin Abstract
Summary: Clifford Ian Simpson (born July 16, 1996), known by his stage name Kevin Abstract, is an American rapper, singer, and producer, best known as the founder and de facto leader of hip-hop group Brockhampton. With Brockhampton, Abstract released eight studio albums from 2017 to 2022. He has released solo work since 2009, and self-released his debut albums MTV1987 (2014) and American Boyfriend: A Suburban Love Story (2016) to significant attention from online blogs and m
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need abstract'}`


Page: Kevin Abstract
Summary: Clifford Ian Simpson (born July 16, 1996), known by his stage name Kevin Abstract, is an American rapper, singer, and producer, best known as the founder and de facto leader of hip-hop group Brockhampton. With Brockhampton, Abstract released eight studio albums from 201

{'input': 'what is the abstract of Attentio is all you need?',
 'output': '**Abstract of “Attention Is All You Need” (Vaswani et\u202fal., 2017)**  \n\n> The dominant sequence‑to‑sequence models are based on complex recurrent or convolutional neural networks in an encoder–decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based entirely on attention mechanisms, with no recurrence and no convolutions. The Transformer achieves state‑of‑the‑art results on translation tasks while allowing for significantly more parallelization during training.  \n> \n> We train a Transformer model on the WMT\u202f2014 English‑to‑German translation task and achieve a BLEU score of 28.4, improving the previous state‑of‑the‑art by 2.6 BLEU points. On the WMT\u202f2014 English‑to‑French translation task, we achieve a BLEU score of 41.0, improving the previous state‑of‑the‑art by 1.